[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/dbs/blob/main/01--postgresql.ipynb)

## Instalacja PostgreSQL

In [1]:
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 10.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../0-cron_3.0pl1-128.1ubuntu1_amd64.deb ...
Unpacking cron (3.0pl1-128.1ubuntu1) ...
Selecting previously unselected package logrotate.
Preparing to unpack .../1-logrotate_3.11.0-0.1ubuntu1_amd64.deb ...
Unpacking logrotate (3.11.0-0.1ubuntu1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../2-netbase_5.4_all.deb ...
Unpacking netbase (5.4) ...
Selecting previously unselected pac

In [ ]:
## 

### Tworzenie użytkownika, bazy oraz przyznawanie uprawnień

In [69]:
!sudo -u postgres psql -c "create user dbuser WITH encrypted password 'dbpw'"

ERROR:  role "datascience" already exists


In [39]:
!sudo -u postgres psql -c "DROP database IF EXISTS db"
!sudo -u postgres psql -c "create database db with OWNER dbuser"
!sudo -u postgres psql db -c "GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO dbuser"

DROP DATABASE
CREATE DATABASE
GRANT


In [22]:
!pip install psycopg2

In [23]:
import psycopg2

In [24]:
connection = psycopg2.connect(user="dbuser",
                                  password="dbpw",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="db")

In [25]:
# Print PostgreSQL version
cursor = connection.cursor()
cursor.execute("SELECT version();")
record = cursor.fetchone()

In [26]:
print(record)

('PostgreSQL 10.19 (Ubuntu 10.19-0ubuntu0.18.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit',)


In [27]:
create_table = '''
DROP TABLE IF EXISTS Employee;
CREATE TABLE IF NOT EXISTS Employee (
  id SERIAL PRIMARY KEY,
  name text NOT NULL,
  sallary money NOT NULL
)
'''

In [28]:
def connect_db():
    connection = psycopg2.connect(user="dbuser",
                                  password="dbpw",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="db")
    return connection

In [29]:
try:
    connection = connect_db()
    cursor = connection.cursor()
    cursor.execute(create_table)
    connection.commit()
except (Exception, psycopg2.DataError) as error:
    print ("Error while creating PostgreSQL table", error)
finally:
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL connection is closed


In [30]:
insert_sql = '''
INSERT INTO Employee (name, sallary) VALUES (%s,%s)
''';

try:
    connection = connect_db()
    cursor = connection.cursor()
    cursor.execute(insert_sql, ("jakub", 2500.00))
    connection.commit()
except (Exception, psycopg2.DataError) as error:
    print ("Error while inserting", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL connection is closed


In [31]:
select = '''
    select * from Employee;
'''

try:
    connection = connect_db()
    cursor = connection.cursor()
    cursor.execute(select)
    records = cursor.fetchall()
except (Exception, psycopg2.DataError) as error:
    print ("Error while inserting", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL connection is closed


In [32]:
print(records)

[(1, 'jakub', '$2,500.00')]


In [33]:
!mkdir -p var/
!wget https://s3.eu-central-1.amazonaws.com/jkan.pl/uek-krakow/finanse-obliczeniowe/retail_sales.gz -O var/retail_sales.gz
!wget https://s3.eu-central-1.amazonaws.com/jkan.pl/uek-krakow/finanse-obliczeniowe/time+series+queries.sql
!sudo -u postgres psql db < var/retail_sales.gz

--2021-11-20 09:42:23--  https://gist.githubusercontent.com/faustofjunqueira/ba97008616148653a9c633c066edaba9/raw/d05a7e8f3568e456417a3f781fbbd35213e529ac/iris-dataset-postgres
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15489 (15K) [text/plain]
Saving to: ‘var/iris.sql’


var/iris.sql          0%[                    ]       0  --.-KB/s               
var/iris.sql        100%[===================>]  15.13K  --.-KB/s    in 0.001s  

2021-11-20 09:42:23 (22.4 MB/s) - ‘var/iris.sql’ saved [15489/15489]



In [ ]:
select = '''
    select * from retail_sales limit 10;
'''

try:
    connection = connect_db()
    cursor = connection.cursor()
    cursor.execute(select)
    records = cursor.fetchall()
except (Exception, psycopg2.DataError) as error:
    print ("Error while inserting", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

In [ ]:
print(records)